# Using Large Language Models to Identify Financial Concepts from 10-K Reports

Open the colab notebook here:

[![](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/calvinw/BusMgmtBenchmarks/blob/main/extract/llm_for_10K_financial_data.ipynb)

Open In Colab

In this notebook we give the idea behind the extraction technique we use
to get financial facts from annual 10-K reports. The main idea involves
calling an LLM with a version of the income statement and the balance
sheet from a 10-K annual report and asking it to identify some financial
concepts we are interested in.

We will use the amazingly useful
[edgartools](https://github.com/dgunning/edgartools) package for
filtering and finding the SEC 10-K filings for our companies.

For each of our companies we are interested in identifying these
concepts from company 10-K reports:

-   Net Revenue
-   Cost of Goods
-   SG&A
-   Operating Profit
-   Net Profit
-   Inventory
-   Current Assets
-   Total Assets
-   Current Liabilities
-   Total Shareholder Equity
-   Total Liabilities and Shareholder Equity

The reason we want to do this is so that we can build a tool that will
allow you to easily compare these values for different companies and for
companies and segments they belong to.

We will find the latest 10-K by using the
[edgartools](https://github.com/dgunning/edgartools) package. The code
behind the library is a very efficient API front end for queries to the
SEC edgar REST api.

Of course once you have a mapping of the labels in the report to the
standard gaap attributes taxonomy for the company you are interested in
you could look the facts labeled with those attribute names on
subsequent reports from the same company. We dont illustrate that but it
makes sense if you need want to optimize looking up this information for
multiple years for the same company and speed and if LLM costs are an
issue.

Our dataset is so small and the LLM costs are minimal for what we want
so we dont pursue that kind of optimization.

In [ ]:
!pip install edgartools
!pip install openai
!pip install pandas

Do some imports and set the display to print out just 10 rows of
dataframes.

In [ ]:
from edgar import set_identity, Company
import pandas as pd
pd.set_option('display.max_rows', 10)

## Using Edgartools to Load the 10K for a Company

You must set your email address in order for edgartools to access the
SEC edgar website. Change “YourEmailAddress” below to be your email
address.

In [ ]:
# Set identity
set_identity("YourEmailAddress")

Next we use the CIK to download the latest 10K for the company. You can
find the CIKs for companies that are American filing at the SEC website.

In [ ]:
# Use the CIK for lululemon 
cik = "1397187"  

# Get the company object using the CIK
company = Company(cik)

# Fetch the 10-K filings
filings = company.get_filings(form="10-K")

# Just get the latest 10-K
filing = filings[0]
filing_obj = filing.obj()

## Getting the Income Statement and the Balance Sheet as Dataframe

First we get the income statement as a dataframe.

In [ ]:
income_df = filing_obj.income_statement.to_dataframe()
income_df

You can see that the current year is in one column and the previous
years after that.

The labels are not a separate column, they are actually “indices” to the
rows, so the data we want is in the first column (with index=0). So we
can drop the columns after that since we only need the current year
data. We use `iloc` to do that as follows:

In [ ]:
income_df = income_df.iloc[:, :1]
income_df

Now we have our labels and the column of data from 2024, the latest in
this case.

This will be cleaner for sending to the LLM, it doesnt need the older
years data

Next we get the balance sheet as a dataframe. Again we just keep the
labels and the first column that has the data we want in it

In [ ]:
balance_df = filing_obj.balance_sheet.to_dataframe()
balance_df = balance_df.iloc[:, :1]
balance_df

## Compare to SEC Filing Web Page

You can compare these to a copy of the report filed by the company with
the SEC. The following html is included in company’s official report to
the SEC and we just reproduce it here so you can compare. Every 10-K has
this kind of html page and its any easy reference for seeing if we
pulled the right values.

In [ ]:
from IPython.display import IFrame

IFrame(src="https://calvinw.github.io/BusMgmtBenchmarks/extract/html/lululemon-2024.html", width=900, height=900)

## Convert the Dataframes to Markdown to Prepare for LLM Call

We convert the dataframes to markdown tables to prepare to send them to
an LLM

We use the `to_markdown` method of the dataframe.

Here is the income statement as markdown:

In [ ]:
income_markdown_table = income_df.to_markdown(index=True, floatfmt=".0f")
print(income_markdown_table)

And the balance sheet as markdown:

In [ ]:
balance_markdown_table = balance_df.to_markdown(index=True, floatfmt=".0f")
print(balance_markdown_table)

## Prepare a Prompt for Passing the Income Statement and Balance Sheet

Now we prepare a prompt to send both the income and balance sheets to
the LLM and ask it to find some financial concepts for us. There are
many variations of this process, you could also give common terms that
you would find these concepts labeled as, or could include the us_gaap
taxonomy names for some of the concepts as well.

In the `edgartools` versions of the the dataframes above there are ways
of getting the gaap attributes that the labels corresponds to for each
of the rows. One could pass those the to LLM as well, to help it
identify the financial concepts that you want, but we dont show that
here.

We just rely on the LLM recognizing the concept from the labels in the
first column of the income statement and the balance report.

Here’s our prompt with our list of concepts and template variables for
the income statement and the balance sheet. This f-string will get
filled in with those.

In [ ]:
prompt = f"""Below is a list of financial concepts (given inside the <concepts>
tags) to be found in the income statement and the balance sheet of a companies
10-K report. 

<concepts>
Net Revenue
Cost of Goods
SG&A
Operating Profit
Net Profit
Inventory
Current Assets
Total Assets
Current Liabilities
Total Shareholder Equity
Total Liabilities and Shareholder Equity
</concepts>

Find the concepts and their values in the income statement or the balance
sheet that follow. 

The income statement is enclosed in  <income_statment> tags. 
The balance sheet is enclosed in <balance_sheet> tags.

Here is the income statement:

<income_statement>
{income_markdown_table}
</income_statement>

Here is the balance sheet:

<balance_sheet>
{balance_markdown_table}
</balance_sheet>

Please return the values found as strings in a json format as follows. Make
sure to provide a number for every concept. Respond only with with the
following format, no other commentary or discussion

The Response Format:
```json
{{
    "Net Revenue": <value>,
    "Cost of Goods": <value>, 
    "SG&A":  <value>,
    "Operating Profit": <value>,
    "Net Profit": <value>,
    "Inventory": <value>,
    "Current Assets": <value>,
    "Total Assets": <value>,
    "Current Liabilities": <value>,
    "Total Shareholder Equity": <value>,
    "Total Liabilities and Shareholder Equity": <value>
}}
```"""

Lets take a look at the prompt before we send it:

In [ ]:
print(prompt)

## Pass the Prompt to an Large Language Model

We use openai compatible call to OpenRouter but you could use any openai
compatible api and key. A good model at the time of writing is
DeepSeek-V3, but many models would work well here. I believe many of the
lightweight 8B models would work. Gemini Flash 1.5 or 2.0, gpt-4o-mini
all work great. Im sure there are many of the more inexpensive models
that would work as well.

### Set Up Your LLM Provider

Paste in the key to your LLM provider as appropriate. You should change
the base_url and the environment variable to be appropriate for your
situation.

As long as the api you are using is openai compatible you should be
fine.

In [ ]:
import os
os.environ["OPENROUTER_API_KEY"] = "paste_your_api_key_here"

Now we call the api. Fix the base_url and api_key. We use DeepSeek V3
but many models would work.

In [ ]:
from openai import OpenAI
import os

try:
    client = OpenAI(
      base_url="https://openrouter.ai/api/v1",
      api_key=os.getenv("OPENROUTER_API_KEY")
    )
    response = client.chat.completions.create(
        model = "deepseek/deepseek-chat",
        messages=[
            {"role": "system", "content": "You are a financial document expert assistant."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.0  # Using 0 temperature for more consistent responses
    )
    llm_response_content = response.choices[0].message.content

except Exception as e:
    print(f"Error occurred: {str(e)}")

print("The LLMs response")
print(llm_response_content)

## Check the Response from the LLM

We could use structred output or json output checking here as well, but
we do not show the details of that.

In [ ]:
print("The income statement again so you can check some of the LLM response: ")
print(income_markdown_table)